See [https://learn.microsoft.com/en-us/azure/bot-service/rest-api/bot-framework-rest-connector-authentication?view=azure-bot-service-4.0&tabs=singletenant](https://learn.microsoft.com/en-us/azure/bot-service/rest-api/bot-framework-rest-connector-authentication?view=azure-bot-service-4.0&tabs=singletenant) for more information on how to authenticate with the Bot Framework Connector service.

In [14]:
import requests
from dotenv import load_dotenv
load_dotenv(override=True)

import os

In [20]:
from azure.identity import ClientSecretCredential

# Get an Azure AD token with client credentials
def get_token():
    credential = ClientSecretCredential(
        tenant_id=os.getenv("BOT_TENANT_ID"),
        client_id=os.getenv("BOT_APP_ID"),
        client_secret=os.getenv("BOT_PASSWORD")
    )
    token = credential.get_token("https://api.botframework.com/.default")
    return token.token


In [18]:
def send_directline_message(conversation_id, message_text, from_user="user1"):
    """
    Sends a message to a DirectLine conversation.

    Args:
        conversation_id (str): The conversation ID to send the message to.
        message_text (str): The message text to send.
        from_user (str): The user ID sending the message.
    Returns:
        dict: The response from the DirectLine API.
    """
    url = f"https://smba.trafficmanager.net/teams/v3/conversations/{conversation_id}/activities"
    directline_secret = get_token()
    headers = {
        "Authorization": f"Bearer {directline_secret}",
        "Content-Type": "application/json"
    }
    payload = {
        "type": "message",
        "from": {"id": from_user},
        "attachments": [
        {
            "contentType": "application/vnd.microsoft.card.adaptive",
            "content": {
                "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
                "type": "AdaptiveCard",
                "version": "1.3",
                "body": [
                    {
                        "type": "TextBlock",
                        "text": "**Hello**, this is a *formatted* message with a [link](https://example.com)!",
                        "wrap": True
                    }
                ]
            }
        }
    ]
    }
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()

In [ ]:
conversation_id = "YOUR_CONVERSATION_ID"  # Replace with your conversation ID

In [ ]:
send_directline_message(conversation_id=conversation_id,
                        message_text="Hello from Python!", from_user="test")

{'id': '1744726365383'}